In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 50)

kv17 = pd.read_csv('KV_17_f.csv', decimal=',')
kv21 = pd.read_csv('KV_21_f.csv', decimal=',')


#Dummyt / One hot encoding

#Dummyt sukupuoli ja 'nykyinen' poliittinen virka
kv17['is_female'] = kv17['Sukupuoli'] - 1
kv21['is_female'] = kv21['Sukupuoli'] - 1

list_of_politi= ['Europarlamentaarikko',
'Kansanedustaja',
'Kunnanvaltuutettu']

kv17[list_of_politi] = kv17[list_of_politi].replace(to_replace=[" ",'1'], value=[0,1])
kv17['is_politician'] = kv17[['Europarlamentaarikko','Kansanedustaja','Kunnanvaltuutettu']].sum(axis=1)
kv17[kv17['is_politician'] == 2] = 1

kv21[list_of_politi] = kv21[list_of_politi].replace(to_replace=[" ",'1'], value=[0,1])
kv21['is_politician'] = kv21[['Europarlamentaarikko','Kansanedustaja','Kunnanvaltuutettu']].sum(axis=1)
kv21[kv21['is_politician'] == 2] = 1

#ehdokkaan äidinkieli
kv17['fi'] = kv17['Ehdokkaan kieli'] == 'FI'
kv17['sv'] = kv17['Ehdokkaan kieli'] == 'SV'
kv21['fi'] = kv21['Ehdokkaan kieli'] == 'FI'
kv21['sv'] = kv21['Ehdokkaan kieli'] == 'SV' 
kv17[['fi','sv']] = kv17[['fi','sv']].astype('int')
kv21[['fi','sv']] = kv21[['fi','sv']].astype('int')

#Valintatietodummy
kv17['chosen'] = kv17['Valintatieto'].map({1:1, 2:0, 3:0})
kv21['chosen'] = kv21['Valintatieto'].map({1:1, 2:0, 3:0})

#drop 0e budgets
kv17 = kv17[kv17['Vaalikampanjan rahoitus yhteensa'] != 0]
kv21 = kv21[kv21['Vaalikampanjan rahoitus yhteensa'] != 0]

#Political party dummies for the major parties
party_dummy = pd.get_dummies(kv17['Puolueen/ryhmän nimilyhenne suomeksi'])
kv17[['KOK','KESK','SDP','VIHR','VAS','PS','RKP','KD']] = party_dummy[['KD    ', 'KESK  ',
       'KOK   ','PS    ','RKP   ','SDP   ','VAS   ', 'VIHR  ']]
party_dummy = pd.get_dummies(kv21['Puolueen/ryhmän nimilyhenne suomeksi'])
kv21[['KOK','KESK','SDP','VIHR','VAS','PS','RKP','KD']] = party_dummy[['KD    ', 'KESK  ',
       'KOK   ','PS    ','RKP   ','SDP   ','VAS   ', 'VIHR  ']]

#Modified df
kv17.head()

In [ ]:
#dtypes seem correct
kv17.dtypes

In [ ]:
#Normalize spending
from sklearn import preprocessing
list_to_normalize = ['Vaalikampanjan kulut yhteensa',
       'Sanoma- ilmaisjakelu- ja aikakauslehdet', 'Radio', 'Televisio',
       'Tietoverkot', 'Muut viestintavalineet', 'Ulkomainonta',
       'Vaalilehtien esitteiden ja muun painetun materiaalin hankinta',
       'Mainonnan suunnittelu', 'Vaalitilaisuudet',
       'Vastikeellisen tuen hankintakulut', 'Muut kulut',
       'Vaalikampanjan rahoitus yhteensa']


kv17[list_to_normalize] = preprocessing.Normalizer().fit_transform(kv17[list_to_normalize])
kv21[list_to_normalize] = preprocessing.Normalizer().fit_transform(kv21[list_to_normalize])
